
### BigData Project on
# NLP for russian language via SparkNLP

Vadim Alperovich <br>
IAD21

[![Go to colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1WpuuDQSxczxT95MxYoOewBnDqKZ9pVOE?authuser=3#scrollTo=TA21Jo5d9SVq)




# 1. Colab-Spark setup

In [1]:
%%time
from IPython.display import clear_output
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

# Install Spark NLP Display for visualization
!pip install --ignore-installed spark-nlp-display
clear_output(wait=True)

CPU times: user 282 ms, sys: 69.6 ms, total: 352 ms
Wall time: 24.1 s


In [2]:
%%time
import sparknlp
import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline

spark = sparknlp.start()

CPU times: user 351 ms, sys: 55 ms, total: 406 ms
Wall time: 13.5 s


# 2. Data loading

In [3]:
from google.colab import drive
drive.mount('/content/drive')
PROJECT_FOLDER = '/content/drive/MyDrive/spark-project/'
%ls {PROJECT_FOLDER}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
bigdata-project  data/


In [4]:
%%time
# download the Lenta.ru v1.0 russian news dataset
# !wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz -P {PROJECT_FOLDER}/data
# download supporting library
!pip install corus
clear_output(wait=True)

CPU times: user 45.5 ms, sys: 18.4 ms, total: 63.9 ms
Wall time: 3.34 s


In [59]:
import json
import corus
import pandas as pd
import numpy as np
import warnings

warnings.simplefilter('ignore')

In [120]:
# %%time
# from corus import load_lenta

# records = load_lenta(path)
# l = []
# for i, r in enumerate(records):
#     if i > 50000:
#         break
#     l.append(list(r))
# df = pd.DataFrame(l, columns=['url', 'title', 'text', 'topic', 'tags', 'date'])
# df.to_csv(f'{PROJECT_FOLDER}/data/lenta-ru-news50000.csv', sep='\t', index=None)

CPU times: user 4.35 s, sys: 257 ms, total: 4.61 s
Wall time: 5.07 s


In [88]:
# %%time
# path = f'{PROJECT_FOLDER}/data/lenta-ru-news.csv.gz'

# df = pd.read_csv(path, nrows=50000, compression='gzip')
# df.head()
# df.to_csv(f'{PROJECT_FOLDER}/data/lenta-ru-news50000.csv', sep='\t', index=None)

In [6]:
spark_df = spark.read \
      .option("inferSchema","true") \
      .option("header", True) \
      .csv(f'{PROJECT_FOLDER}/data/lenta-ru-news50000.csv', sep='\t')
print(f'Hell yeah, spark_df is {type(spark_df)}!')
print(f'spark_df has {spark_df.count()} rows')
spark_df

Hell yeah, spark_df is <class 'pyspark.sql.dataframe.DataFrame'>!
spark_df has 50001 rows


DataFrame[url: string, title: string, text: string, topic: string, tags: string, date: string]

In [7]:
spark_df.show(n=10, truncate=30)

+------------------------------+------------------------------+------------------------------+-----------------+---------------+----+
|                           url|                         title|                          text|            topic|           tags|date|
+------------------------------+------------------------------+------------------------------+-----------------+---------------+----+
|https://lenta.ru/news/2018/...|Названы регионы России с са...|Вице-премьер по социальным ...|           Россия|       Общество|null|
|https://lenta.ru/news/2018/...|Австрия не представила дока...|Австрийские правоохранитель...|            Спорт|    Зимние виды|null|
|https://lenta.ru/news/2018/...|Обнаружено самое счастливое...|Сотрудники социальной сети ...|      Путешествия|            Мир|null|
|https://lenta.ru/news/2018/...|В США раскрыли сумму расход...|С начала расследования росс...|              Мир|       Политика|null|
|https://lenta.ru/news/2018/...|Хакеры рассказали о планах ...

# 3. EDA & Preprocessing

In [9]:
spark_df = spark_df.filter(F.col('text').isNotNull() & F.col('topic').isNotNull() & F.col('tags').isNotNull())
spark_df = spark_df.filter(F.col('text') != '')
print(f'Count after filtering: {spark_df.count()}')

Count after filtering: 49129


In [10]:
column = 'topic'
topic_unique = spark_df.select(column).distinct().count()
print(f'*{column}* columns has {topic_unique} unique values', )
spark_df.select(column).groupBy(column).count().orderBy(F.desc('count')).show(n=30)

*topic* columns has 18 unique values
+-----------------+-----+
|            topic|count|
+-----------------+-----+
|              Мир| 6896|
|           Россия| 6830|
|            Спорт| 4912|
|        Экономика| 4716|
|   Интернет и СМИ| 3886|
|  Наука и техника| 3511|
|         Из жизни| 3320|
|      Бывший СССР| 3214|
|         Культура| 3169|
|Силовые структуры| 2681|
|              Дом| 2088|
|         Ценности| 2056|
|      Путешествия| 1197|
|   69-я параллель|  423|
|             Крым|  150|
|    Культпросвет |   77|
|           Бизнес|    2|
|           Оружие|    1|
+-----------------+-----+



In [11]:
column = 'tags'
tags_unique = spark_df.select(column).distinct().count()
print(f'*{column}* columns has {tags_unique} unique values', )
spark_df.select(column).groupBy(column).count().orderBy(F.desc('count')).show(n=30)

*tags* columns has 79 unique values
+--------------------+-----+
|                tags|count|
+--------------------+-----+
|            Политика| 5521|
|            Общество| 4501|
|        Происшествия| 2933|
|              Футбол| 2798|
|             Украина| 2548|
|        Госэкономика| 2390|
|            Интернет| 1987|
|                Кино| 1486|
|     Следствие и суд| 1473|
|            Квартира| 1371|
|              Музыка| 1229|
|                Люди| 1177|
|              Оружие| 1144|
|              Бизнес|  882|
|             Регионы|  851|
|               Наука|  820|
|         Зимние виды|  816|
|               Звери|  814|
|           Конфликты|  770|
|             Явления|  729|
|              Космос|  699|
|          ТВ и радио|  696|
|          Бокс и ММА|  682|
|            Криминал|  573|
|               Стиль|  571|
|      Деловой климат|  562|
|             События|  554|
|        Преступность|  539|
|             Coцсети|  527|
|Полиция и спецслужбы|  512|
+------

In [12]:
def get_token_count(text, token_len=False):
    if token_len:
        return len(text.split(' '))
    return len(text)

spark_get_char_count = F.udf(lambda x: get_token_count(x), T.IntegerType())
spark_get_token_count = F.udf(lambda x: get_token_count(x, token_len=True), T.IntegerType())

In [13]:
%%time
# ```text``` stats
spark_df = spark_df.withColumn("char_count", spark_get_char_count(F.col("text")))
spark_df = spark_df.withColumn("token_count", spark_get_token_count(F.col("text")))
# ```title``` stats
spark_df = spark_df.withColumn("char_count_title", spark_get_char_count(F.col("title")))
spark_df = spark_df.withColumn("token_count_title", spark_get_token_count(F.col("title")))

CPU times: user 29.2 ms, sys: 9.35 ms, total: 38.6 ms
Wall time: 224 ms


In [14]:
spark_df.show(n=5, truncate=10)

+----------+----------+----------+----------+----------+----+----------+-----------+----------------+-----------------+
|       url|     title|      text|     topic|      tags|date|char_count|token_count|char_count_title|token_count_title|
+----------+----------+----------+----------+----------+----+----------+-----------+----------------+-----------------+
|https:/...|Названы...|Вице-пр...|    Россия|  Общество|null|       660|         92|              58|                7|
|https:/...|Австрия...|Австрий...|     Спорт|Зимние ...|null|      1072|        137|              65|                6|
|https:/...|Обнаруж...|Сотрудн...|Путешес...|       Мир|null|       961|        120|              44|                5|
|https:/...|В США р...|С начал...|       Мир|  Политика|null|      1347|        189|              65|                8|
|https:/...|Хакеры ...|Хакерск...|       Мир|  Общество|null|      2066|        255|              66|                6|
+----------+----------+----------+------

In [15]:
spark_df.select('char_count', 'token_count', 'char_count_title', 'token_count_title').describe().show()

+-------+------------------+------------------+------------------+------------------+
|summary|        char_count|       token_count|  char_count_title| token_count_title|
+-------+------------------+------------------+------------------+------------------+
|  count|             49129|             49129|             49129|             49129|
|   mean| 1265.777198803151|173.11160414419183| 54.30859573775163| 6.301736245394777|
| stddev|470.70315275350913| 63.12842848537941|13.135353121587741|1.4749514018188228|
|    min|               252|                31|                12|                 1|
|    max|              9282|              1246|                84|                12|
+-------+------------------+------------------+------------------+------------------+



# 4. Pipelines & Experiments

## 4.1 Text processing pipeline

In [16]:
%%time 

document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

normalizer = Normalizer() \
      .setInputCols(["token"]) \
      .setLowercase(True) \
      .setOutputCol("normalized_token")

stopwords_cleaner = StopWordsCleaner.pretrained("stopwords_ru", "ru")\
      .setInputCols("normalized_token")\
      .setOutputCol("clean_token")\
      .setCaseSensitive(False)

lemmatizer = LemmatizerModel.pretrained("lemma", "ru") \
        .setInputCols(["clean_token"]) \
        .setOutputCol("lemma")

finisher = Finisher() \
      .setInputCols(["lemma"]) \
      .setOutputCols(["token_features"]) \
      .setOutputAsArray(True) \
      .setCleanAnnotations(False)

preprocessing_pipeline = Pipeline(stages=[
                                  document_assembler, 
                                  tokenizer,
                                  normalizer,
                                  stopwords_cleaner,
                                  lemmatizer,
                                  finisher])

stopwords_ru download started this may take some time.
Approximate size to download 2.9 KB
[OK!]
lemma download started this may take some time.
Approximate size to download 1.3 MB
[OK!]
CPU times: user 211 ms, sys: 38 ms, total: 249 ms
Wall time: 18.6 s


In [117]:
%%time
preprocessor_model = preprocessing_pipeline.fit(spark_df[['text']])
doc_df = preprocessor_model.transform(spark_df[['text']])
doc_df.printSchema()

root
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- token: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 

In [118]:
doc_df[['text', 'token_features']].show(n=10, truncate=50)

+--------------------------------------------------+--------------------------------------------------+
|                                              text|                                    token_features|
+--------------------------------------------------+--------------------------------------------------+
|Вице-премьер по социальным вопросам Татьяна Гол...|[вицепремьер, социальный, вопрос, татьяна, голи...|
|Австрийские правоохранительные органы не предст...|[австрийский, правоохранительный, орган, предст...|
|Сотрудники социальной сети Instagram проанализи...|[сотрудник, социальный, сеть, instagram, анализ...|
|С начала расследования российского вмешательств...|[расследование, российский, вмешательство, выбо...|
|Хакерская группировка Anonymous опубликовала но...|[хакерская, группировка, anonymous, опубликоват...|
|Архиепископ канонической Украинской православно...|[архиепископ, канонической, украинский, правосл...|
|Российская молодежь лучше усвоит духовные ценно...|[российский,

In [44]:
# doc_df.withColumn(
#     "tmp", 
#     F.explode("lemma"))\
#     .select("tmp.*")\
#     .show(truncate=False)

# 4.2 Bag-of-Words + LogReg

In [17]:
from pyspark.ml.feature import CountVectorizer, HashingTF, IDF, OneHotEncoder, StringIndexer, VectorAssembler, SQLTransformer

In [18]:
countVectors = CountVectorizer(inputCol="token_features", outputCol="features", vocabSize=5000, minDF=15)

label_stringIdx = StringIndexer(inputCol="topic", outputCol="label")

bow_pipeline = Pipeline(
    stages=preprocessing_pipeline.getStages()+[countVectors, label_stringIdx])
bow_pipeline.getStages()

[DocumentAssembler_eb4295413d1c,
 Tokenizer_3a1695ba49e4,
 Normalizer_7f7548997923,
 STOPWORDS_CLEANER_3187062cd9db,
 LEMMATIZER_59ad4a12c2e7,
 Finisher_545f9675b8de,
 CountVectorizer_8ef60df81603,
 StringIndexer_5f2f27402673]

In [19]:
%%time
bow_model = bow_pipeline.fit(spark_df[['text', 'topic']])
bow_df = bow_model.transform(spark_df[['text', 'topic']])

CPU times: user 1.39 s, sys: 226 ms, total: 1.62 s
Wall time: 2min 38s


In [20]:
bow_df.select('token_features', 'features', 'label', 'topic').show(truncate=50)

+--------------------------------------------------+--------------------------------------------------+-----+-----------------+
|                                    token_features|                                          features|label|            topic|
+--------------------------------------------------+--------------------------------------------------+-----+-----------------+
|[вицепремьер, социальный, вопрос, татьяна, голи...|(5000,[0,1,3,13,14,15,26,36,54,63,81,83,84,85,1...|  1.0|           Россия|
|[австрийский, правоохранительный, орган, предст...|(5000,[1,2,5,11,22,40,57,61,62,70,101,113,115,1...|  2.0|            Спорт|
|[сотрудник, социальный, сеть, instagram, анализ...|(5000,[0,5,14,30,32,48,52,65,86,91,115,123,168,...| 12.0|      Путешествия|
|[расследование, российский, вмешательство, выбо...|(5000,[0,2,6,7,15,16,19,26,31,33,48,60,61,80,82...|  0.0|              Мир|
|[хакерская, группировка, anonymous, опубликоват...|(5000,[1,2,5,10,22,26,30,32,39,43,55,58,60,64,6...| 

In [21]:
%%time
# set seed for reproducibility
(train_df, test_df) = bow_df.select('features', 'label').randomSplit([0.7, 0.3], seed=100)

CPU times: user 6.13 ms, sys: 29 µs, total: 6.16 ms
Wall time: 70.5 ms


In [ ]:
print(f'Training Dataset Count: {train_df.count()}')
print(f'Test Dataset Count: {test_df.count()}')

### LogReg

In [24]:
from pyspark.ml.classification import LogisticRegression

In [28]:
%%time

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(train_df)
predictions = lrModel.transform(test_df)

CPU times: user 1.86 s, sys: 269 ms, total: 2.12 s
Wall time: 4min 45s


In [29]:
predictions.show(10, truncate=5)

+--------+-----+-------------+-----------+----------+
|features|label|rawPrediction|probability|prediction|
+--------+-----+-------------+-----------+----------+
|   (5...|  3.0|        [2...|      [0...|       3.0|
|   (5...|  0.0|        [3...|      [0...|       0.0|
|   (5...|  0.0|        [5...|      [0...|       0.0|
|   (5...|  0.0|        [2...|      [0...|       0.0|
|   (5...|  0.0|        [5...|      [0...|       0.0|
|   (5...|  0.0|        [6...|      [0...|       0.0|
|   (5...|  0.0|        [6...|      [0...|       0.0|
|   (5...|  0.0|        [4...|      [0...|       0.0|
|   (5...|  0.0|        [4...|      [0...|       0.0|
|   (5...|  0.0|        [4...|      [0...|       0.0|
+--------+-----+-------------+-----------+----------+
only showing top 10 rows



In [30]:
%%time
from sklearn.metrics import confusion_matrix, classification_report, f1_score
y_true = predictions.select("label")
y_true = y_true.toPandas()

y_pred = predictions.select("prediction")
y_pred = y_pred.toPandas()

CPU times: user 2.37 s, sys: 409 ms, total: 2.78 s
Wall time: 4min 43s


In [57]:
label_names = bow_model.stages[-1].labels
label_names =  [l  for i, l in enumerate(label_names) if i in y_true.label.unique()]
len(labels)

18

In [60]:
print(classification_report(y_true.label, y_pred.prediction, target_names=label_names))

                   precision    recall  f1-score   support

              Мир       0.78      0.90      0.84      2081
           Россия       0.68      0.83      0.74      1973
            Спорт       0.94      0.97      0.95      1487
        Экономика       0.85      0.87      0.86      1435
   Интернет и СМИ       0.80      0.78      0.79      1186
  Наука и техника       0.88      0.86      0.87      1016
         Из жизни       0.80      0.80      0.80      1015
      Бывший СССР       0.86      0.77      0.81       929
         Культура       0.89      0.88      0.89       942
Силовые структуры       0.82      0.70      0.75       841
              Дом       0.92      0.77      0.84       601
         Ценности       0.97      0.79      0.87       629
      Путешествия       0.88      0.50      0.64       363
   69-я параллель       0.75      0.03      0.05       119
             Крым       1.00      0.04      0.08        49
    Культпросвет        0.00      0.00      0.00       

In [63]:
print(classification_report(y_true.label, y_pred.prediction, target_names=label_names, labels=range(0, 12)))

                   precision    recall  f1-score   support

              Мир       0.78      0.90      0.84      2081
           Россия       0.68      0.83      0.74      1973
            Спорт       0.94      0.97      0.95      1487
        Экономика       0.85      0.87      0.86      1435
   Интернет и СМИ       0.80      0.78      0.79      1186
  Наука и техника       0.88      0.86      0.87      1016
         Из жизни       0.80      0.80      0.80      1015
      Бывший СССР       0.86      0.77      0.81       929
         Культура       0.89      0.88      0.89       942
Силовые структуры       0.82      0.70      0.75       841
              Дом       0.92      0.77      0.84       601
         Ценности       0.97      0.79      0.87       629

        micro avg       0.82      0.84      0.83     14135
        macro avg       0.85      0.83      0.83     14135
     weighted avg       0.83      0.84      0.83     14135



---

## 4.3 Transfer learning